In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import time

%matplotlib inline

#每次可以输出多个变量
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from pylab import rcParams
rcParams['figure.figsize'] = 14, 6

#中文字体
import matplotlib  
matplotlib.use('qt4agg')  
#指定默认字体  
matplotlib.rcParams['font.sans-serif'] = ['SimHei']   
matplotlib.rcParams['font.family']='sans-serif'  
#解决负号'-'显示为方块的问题  
matplotlib.rcParams['axes.unicode_minus'] = False 

/usr/local/miniconda2/envs/py35/lib/python3.5/site-packages/matplotlib/__init__.py:1405: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [3]:
cols = pickle.load(open('../cjy/input/test_matrix/m_622_cols.p','rb'))
cols[:5]
coo = pickle.load(open('../cjy/input/test_matrix/m_622.p','rb'))
coo = coo.tocoo(copy=False)
m_622_frame = pd.DataFrame({'index': coo.row, 'col': coo.col, 'data': coo.data}
                 )[['index', 'col', 'data']].sort_values(['index', 'col']
                 ).reset_index(drop=True)
m_622_frame.head()

['full_order', 'b_3758640', 'b_3155503', 'b_56331008', 'b_33489263']

,index,col,data
0,0,0,1138172.0
1,0,948,-82.0
2,0,1483,-85.0
3,0,2105,-61.0
4,0,3453,-77.0


In [2]:
def mall_wifi_dateframe(mall_id, path):
    cols = pickle.load(open('../cjy/input/'+path+'/'+mall_id+'_cols.p','rb'))
    coo = pickle.load(open('../cjy/input/'+path+'/'+mall_id+'.p','rb'))
    coo = coo.tocoo(copy=False)
    coo_group = pd.DataFrame({'row': coo.row, 'col': coo.col, 'data': coo.data}
                 )[['row', 'col', 'data']].sort_values(['row', 'col']
                 ).reset_index(drop=True).groupby('col')
    df = pd.DataFrame(0 * np.ones((max(coo.row)+1, len(cols))), columns=cols, dtype=np.int64)
    tmp = coo_group.apply(lambda x:x.row.values)
    for col,rows,values in zip(tmp.index.tolist(),tmp.values,
                                coo_group.apply(lambda x:x.data.tolist())):
        df.iloc[rows,col]=values
    return df

In [69]:
mall = mall_wifi_dateframe('m_622', 'train_matrix')
mall.head()

,full_order,b_3758640,b_3155503,b_56331008,b_33489263,b_18655345,b_9091070,b_6724355,b_49662085,b_19188542,...,b_18304278,b_41824143,b_55555065,b_23622024,b_19054820,b_20952577,b_39241838,b_40230368,b_53654217,b_56549844
0,26,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,209,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,299,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,403,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1152,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
def shop_sim(mall_frame, x, y):
    temp1 = mall_frame[mall_frame['index'] == x].index
    temp2 = mall_frame[mall_frame['index'] == y].index
    dot = 0
    xx = 0
    for i in temp1:
        yy = 0
        if mall_frame.iloc[i].col != 0:
            x_stre = mall_frame.iloc[i]['data']
            xx = xx + x_stre**2
            for j in temp2:
                if mall_frame.iloc[j].col == mall_frame.iloc[i].col:
                    y_stre = mall_frame.iloc[j]['data']
                    yy = yy + y_stre**2
                    dot = dot + x_stre * y_stre
    if dot == 0:
        return 0
    else:
        return float((1000 * dot)/(xx * yy))

In [3]:
import math
import pickle
from tqdm import tqdm

In [3]:
def Cosine(x, y):
    x = np.array(x)
    y = np.array(y)
    return float(x.dot(y)/(math.sqrt((x**2).sum()) * math.sqrt((y**2).sum())))

In [73]:
pd.DataFrame(1-pairwise_distances(mall.drop(['full_order'], axis=1).as_matrix(), metric="cosine"))

,0,1,2,3,4,5,6,7,8,9,...,10594,10595,10596,10597,10598,10599,10600,10601,10602,10603
0,1.000000,0.000000,0.000000,0.000000,0.000000,0.435185,0.000000,0.000000,0.000000,0.000000,...,0.0,0.190643,0.471624,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.104315,0.000000,...,0.0,0.109011,0.000000,0.000000,0.184054,0.000000,0.000000,0.000000,0.000000,0.559009
3,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.391835,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.300876,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.435185,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.201020,0.497017,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.000000,0.000000,0.000000,0.391835,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.116861,0.112273,0.213947,0.000000,0.000000
8,0.000000,0.000000,0.104315,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,...,0.0,0.110610,0.000000,0.000000,0.077633,0.000000,0.000000,0.000000,0.000000,0.000000
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [9]:
train_shop = pd.read_csv('../input/训练数据-ccf_first_round_shop_info.csv')
train_user = pd.read_csv('../input/训练数据-ccf_first_round_user_shop_behavior.csv')
test_shop = pd.read_csv('../input/AB榜测试集-evaluation_public.csv')

train_shop.shape
train_shop.head()
train_user.shape
train_user.head()
test_shop.shape
test_shop.head()

(8477, 6)

,shop_id,category_id,longitude,latitude,price,mall_id
0,s_26,c_4,122.346736,31.833507,57,m_690
1,s_133,c_6,121.134362,31.197511,58,m_6587
2,s_251,c_38,121.000505,30.907667,34,m_5892
3,s_372,c_30,119.864982,26.659876,44,m_625
4,s_456,c_26,122.594243,31.581499,44,m_3839


(1138015, 6)

,user_id,shop_id,time_stamp,longitude,latitude,wifi_infos
0,u_376,s_2871718,2017-08-06 21:20,122.308291,32.088040,b_6396480|-67|false;b_41124514|-86|false;b_287...
1,u_376,s_2871718,2017-08-06 21:20,122.308162,32.087970,b_6396480|-67|false;b_56328155|-73|false;b_411...
2,u_1041,s_181637,2017-08-02 13:10,117.365255,40.638214,b_8006367|-78|false;b_2485110|-52|false;b_3005...
3,u_1158,s_609470,2017-08-13 12:30,121.134451,31.197416,b_26250579|-73|false;b_26250580|-64|false;b_26...
4,u_1654,s_3816766,2017-08-25 19:50,122.255867,31.351320,b_39004150|-66|false;b_39004148|-58|false;b_21...


(483931, 7)

,row_id,user_id,mall_id,time_stamp,longitude,latitude,wifi_infos
0,118742,u_30097142,m_3916,2017-09-05 13:00,122.141011,39.818847,b_34366982|-82|false;b_37756289|-53|false;b_41...
1,118743,u_30097803,m_5085,2017-09-06 13:10,118.191907,32.855858,b_36722251|-81|false;b_10537579|-75|false;b_43...
2,118744,u_30097889,m_4033,2017-09-06 17:40,119.192110,32.424667,b_30026291|-74|false;b_30026290|-74|false;b_36...
3,118745,u_30098996,m_4515,2017-09-03 12:10,120.612201,34.055249,b_33412374|-77|false;b_22084893|-86|false;b_52...
4,118746,u_30099170,m_7168,2017-09-02 20:40,116.861989,40.326858,b_19882704|-77|false;b_2241462|-49|false;b_585...


In [10]:
train = train_user.merge(train_shop, left_on='shop_id', right_on='shop_id', how='left')

In [11]:
train_mall_ids = set(train_shop.mall_id)
test_mall_ids = set(test_shop.mall_id)
userShop.head()

del train_shop
del train_user
del test_shop

0    s_2871718
1    s_2871718
2     s_181637
3     s_609470
4    s_3816766
Name: shop_id, dtype: object

In [6]:
from sklearn.metrics import pairwise_distances
from scipy.spatial.distance import cosine
from scipy.sparse import csc_matrix

In [ ]:
for mall_id in tqdm(train_mall_ids):
    mall = mall_wifi_dateframe(mall_id, 'train_matrix')
    temp = pd.DataFrame(1-pairwise_distances(mall.drop(['full_order'], axis=1).as_matrix(), metric="cosine"))
    full_order = mall.full_order
    shop_frame = pd.DataFrame()
    
    for i in range(temp.shape[0]):
        k = 8
        p = 0
        q = 0
        shop = []
        shop_id = userShop[full_order[i]]
        shop.append(shop_id)
        index = temp.iloc[i].sort_values(ascending=False).index
        while p < k:
            if (userShop[full_order[index[q]]] != shop_id) & (userShop[full_order[index[q]]] not in shop):
                shop.append(userShop[full_order[index[q]]])
                q = q + 1
                p = p + 1
            else:
                q = q + 1
        shop_frame = pd.concat([shop_frame, pd.DataFrame(shop)], axis=1)
        
    print(mall_id)
    pickle.dump(shop_frame, open('./output/train_shop/' + str(mall_id) + '.p', 'wb'), protocol=4)

  1%|          | 1/97 [02:08<3:25:10, 128.24s/it]

m_3445


  2%|▏         | 2/97 [05:50<4:37:38, 175.35s/it]

m_2578


  3%|▎         | 3/97 [08:33<4:28:22, 171.30s/it]

m_1089
